In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_black
%load_ext nb_mypy

In [ ]:
import nbimporter
from Exercise04AI import Exercise04AI
from Exercise05AI import Exercise05AI

# Aufgabe 06: Minimax mit Alpha-Beta-Pruning und Memoisierung

Dieses Notebook implementiert den Minimax-Algorithmus mit Alpha-Beta-Pruning und Memoisierung.

In [ ]:
import chess
from typing import Any


class Exercise06AI(Exercise05AI):
    """Chooses middle game moves using minimax algorithm, alpha-beta-pruning and memoization"""

    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        self.cache: dict[tuple, Any] = {}

    def reset(self) -> None:
        """Resets all internal variables"""
        super().reset()
        self.cache.clear()

## Memoisierungsfunktion

Die Klasse wird um eine Variable `cache` erweitert. Initial ist `cache` ein leeres Dictionary, welches als Schlüssel ein Tupel annimmt und als Wert ein beliebiges Objekt zurückgibt.

Die Memoisierungsfunktion `memoize_minimax` nimmt die Minimax-Funktion `minimax` als Argument und gibt eine memoisierte Version `minimax_memoized` dieser Funktion zurück.

Die memoisierte Funktion `minimax_memoized` versucht zunächst, den Rückgabewert der Ursprungsfunktion aus dem `cache` auszulesen. Dazu wird die Hilsfunktion `get_from_cache` verwendet. Beinhaltet der `cache` ein Ergebnis für die gegebenen Argumente `args`, so wird dieses zurückgegeben. Andernfalls wird die Ursprungsfunktion `minimax` aufgerufen, der Rückgabewert wird für die verwendeten Argumente im `cache` hinterlegt und schlussendlich wird der berechnete Wert zurückgegeben. Zum Speichern des neuen Rückgabewertes wird die Hilfsfunktion `store_in_cache` verwendet.  
Der Cache wird bei jedem Bauernzug, Schlagen einer Figur und veränderten Rochade- oder En-Passant-Rechten zurückgesetzt, da in diesem Fall die zuvor berechneten Stellungen nicht wieder auftreten können.

Da die Memoisierungsfunktion in den weiteren ChessAI-Versionen ebenfalls verwendet werden soll, wird diese als Dekorator-Funktion implementiert.

### Cache-Verwaltung

Im Vergleich zu `Exercise04AI` sind die Argumente $alpha$ und $beta$, welche für das Alpha-Beta-Pruning benötigt werden, zum Funktionsaufruf der Methode `minimax` hinzugekommen. Demzufolge wird die Interaktion mit dem Cache überarbeitet.

#### Speicherung im Cache

Bisher war `result` ein Tupel der Form $$result = (evaluation, move)$$
Um Alpha-Beta-Pruning memoisieren zu können, wird nun nicht mehr `result` im Cache gespeichert, sondern ein Tripel der Form $$ value = (flag, evaluation, move) $$ mit $$ flag \in \{ \; \text{'≤'}, \text{'='}, \text{'≥'} \; \} $$

wobei gilt $$flag = \begin{cases}
\text{'≤'} \; \text{wenn} \; evaluation \le alpha\\
\text{'='} \; \text{wenn} \; evaluation < beta\\
\text{'≥'} \; \text{sonst}
\end{cases}$$

Das Speichern dieses Tupels wird von der Funktion `store_in_cache` übernommen. Diese bekommt als Argumente den Cache-Schlüssel des aktuellen Boards (`key`), das berechnete Minimax-Ergebnis als Tupel (`result`), die Grenzwerte $alpha$ und $beta$ und speichert das Ergebnis nach dem oben gegebenen Verfahren im Cache ab.

#### Abrufen aus dem Cache

Die Funktion `get_from_cache` nimmt als Parameter die Funktion `minimax`, das den Cache-Schlüssel für das aktuelle Board (`key`), sowie alle Parameter der Funktion `minimax` entgegen und gibt ein Tupel der Form $$(evaluation, move)$$ zurück. Zunächst wird das Tripel $$ value = (flag, evaluation, move) $$ mit dem gegebenen `key` aus dem Cache extrahiert. Es folgt eine Fallunterscheidung nach dem Wert von $flag$:

1. Fall: $flag = \text{'='}$

    In diesem Fall wird das Tupel $(evaluation, move)$ zurückgegeben.

2. Fall: $flag = \text{'≤'}$

    Ist $evaluation \le alpha$, so kann das Tupel $(evaluation, move)$ direkt zurückgegeben werden. Ist $evaluation < beta$, so wird die Funktion `minimax` mit $alpha$ und $evaluation$ aufgerufen, andernfalls gilt $evaluation \ge beta$ und die Funktion `minimax` wird mit den ursprünglichen Grenzen $alpha$ und $beta$ aufgerufen. Das Ergebnis wird im Cache gespeichert und zurückgegeben.

3. Fall: Trifft keiner der vorherigen Fälle zu, muss $flag = \text{'≥'}$ gelten und es folgt eine erneute Fallunterscheidung:

    Ist $evaluation \le alpha$, so wird die Funktion `minimax` mit $alpha$ und $beta$ aufgerufen. Ist $evaluation < beta$, so wird `minimax` stattdessen mit $evaluation$ und $beta$ aufgerufen. Das Ergebnis wird jeweils im Cache gespeichert und zurückgegeben.

    Trifft keiner der beiden Fälle zu, muss $evaluation > beta$ gelten und das Tupel $(evaluation, move)$ kann direkt zurückgegeben werden.


In [ ]:
from typing import Any, Callable
import sys


class Exercise06AI(Exercise06AI):  # type: ignore
    @staticmethod
    def memoize_minimax(minimax: Callable):
        def minimax_memoized(
            self,
            board: chess.Board,
            depth: int,
            current_evaluation: int,
            alpha: int = -Exercise06AI.LIMIT,
            beta: int = Exercise06AI.LIMIT,
        ):
            key = Exercise04AI.get_key(board, depth)
            self.stats[-1]["cache_tries"] += 1
            if key in self.cache:
                self.stats[-1]["cache_hits"] += 1
                return self.get_from_cache(
                    minimax, key, board, depth, current_evaluation, alpha, beta
                )
            result = minimax(self, board, depth, current_evaluation, alpha, beta)
            self.store_in_cache(key, result, alpha, beta)
            return result

        return minimax_memoized

    @memoize_minimax
    def minimax(self, *args) -> int:
        """Memoized version of the Exercise05AI minimax implementation"""
        return super().minimax(*args)

    def store_in_cache(self, key: tuple, result: tuple, alpha: int, beta: int) -> None:
        """Stores the result of a minimax computation in the cache."""
        evaluation, move = result
        if evaluation <= alpha:
            self.cache[key] = ("≤", evaluation, move)
        elif evaluation < beta:
            self.cache[key] = ("=", evaluation, move)
        else:
            self.cache[key] = ("≥", evaluation, move)

    def get_from_cache(
        self,
        minimax: Callable,
        key: tuple,
        board: chess.Board,
        depth: int,
        current_eval: int,
        alpha: int,
        beta: int,
    ) -> tuple:
        """Gets a result from the cache if possible."""
        flag, evaluation, move = self.cache[key]
        if flag == "=":
            return evaluation, move
        elif flag == "≤":
            if evaluation <= alpha:
                return evaluation, move
            elif evaluation < beta:
                result = minimax(self, board, depth, current_eval, alpha, evaluation)
                self.store_in_cache(key, result, alpha, evaluation)
                return result
            else:
                result = minimax(self, board, depth, current_eval, alpha, beta)
                self.store_in_cache(key, result, alpha, beta)
                return result
        else:
            if evaluation <= alpha:
                result = minimax(self, board, depth, current_eval, alpha, beta)
                self.store_in_cache(key, result, alpha, beta)
                return result
            elif evaluation < beta:
                result = minimax(self, board, depth, current_eval, evaluation, beta)
                self.store_in_cache(key, result, evaluation, beta)
                return result
            else:
                return evaluation, move

    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move:
        """Gets the best next move"""
        self.stats[-1]["cache_tries"] = 0
        self.stats[-1]["cache_hits"] = 0
        next_move = super().get_next_middle_game_move(board)
        if board.is_irreversible(next_move):
            self.cache.clear()
            self.stats[-1]["cache_cleared"] = True
        else:
            self.stats[-1]["cache_cleared"] = False
        self.stats[-1]["cache_size_mb"] = round(
            sys.getsizeof(self.cache) / (1024 * 1024), 2
        )
        return next_move

## Debugging Bereich

Die folgenden Zellen enthalten Unit-Tests der oben implementierten Funktionen.

In [ ]:
from AIBaseClass import ChessAI
import Exercise03AI as Exercise03Tests
import Exercise04AI as Exercise04Tests

In [ ]:
# Create player and board
unit_test_player = Exercise06AI(player_name="Ex06AI", search_depth=2)
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
next_move = chess.Move.from_uci("d4c3")  # white queen capture
board

In [ ]:
# Test cache store function
def test_store_in_cache(unit_test_player: ChessAI, board: chess.Board):
    unit_test_player.cache = {}  # Clear cache
    test_result = (10, next_move)

    test_key_1 = ("test", 1)
    unit_test_player.store_in_cache(test_key_1, test_result, alpha=5, beta=15)
    assert unit_test_player.cache[test_key_1] == (
        "=",
        *test_result,
    ), "Cache result does not match expected value!"

    test_key_2 = ("test", 2)
    unit_test_player.store_in_cache(test_key_2, test_result, alpha=12, beta=15)
    assert unit_test_player.cache[test_key_2] == (
        "≤",
        *test_result,
    ), "Cache result does not match expected value!"

    test_key_3 = ("test", 3)
    unit_test_player.store_in_cache(test_key_3, test_result, alpha=5, beta=8)
    assert unit_test_player.cache[test_key_3] == (
        "≥",
        *test_result,
    ), "Cache result does not match expected value!"

    print(f"Cache: {unit_test_player.cache}")

In [ ]:
test_store_in_cache(unit_test_player, board)

In [ ]:
# Test minimax
Exercise04Tests.test_minimax(unit_test_player, board)

In [ ]:
# Test minimax again (now memoized)
Exercise04Tests.test_memoized_minimax(unit_test_player, board)

In [ ]:
# Test next move function (with memoized minimax result)
Exercise03Tests.test_next_move(unit_test_player, board)

In [ ]:
# Test reset function
Exercise04Tests.test_reset(unit_test_player, board)

## Temporärer Bereich

Der folgende Bereich dient zum temporären Debuggen und kann nicht-funktionierenden Code enthalten. Dieser Bereich wird vor der Abgabe entfernt.